In [1]:
from torch.utils.data import DataLoader
from Dataset.NSRVideoDataset import NSRVideoDataset
from easydict import EasyDict
import torch
from torchvision.transforms import transforms
from torch.utils.tensorboard import SummaryWriter
import time
import os
import torchsummary
from tqdm import tqdm
import pandas as pd
import numpy as np

In [2]:
args = EasyDict({
    'dataset_path' : r"D:\Video-Dataset\2022-NSR-3m-재분류", # root directory path
    'split' : (0.8, 0.1), # train/validation, validation/train
    'dataset_type' : ("Train", "Validation", "Test"),
    'batch_size' : 32,
    'epochs' : 50,
    'learning_rate' : 1e-3,
    'model_name' : "Resnet101",
})

In [3]:
train_transform = transforms.Compose([
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(), 
    transforms.ToTensor(), 
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

etc_transform = transforms.Compose([
    transforms.Resize(224), 
    transforms.ToTensor(), 
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

train_dataset = NSRVideoDataset(args.dataset_path, args.split, args.dataset_type[0], train_transform)
train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True, pin_memory=True, num_workers=1, drop_last=False)

validation_dataset = NSRVideoDataset(args.dataset_path, args.split, args.dataset_type[1], etc_transform)
validation_dataloader = DataLoader(validation_dataset, batch_size=args.batch_size, shuffle=False, pin_memory=True, num_workers=1, drop_last=False)

test_dataset = NSRVideoDataset(args.dataset_path, args.split, args.dataset_type[2], etc_transform)
test_dataloader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False, pin_memory=True, num_workers=1, drop_last=False)

In [4]:
from Models.VGG16 import VGG16
from Models.Resnet101 import Resnet101

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = Resnet101().to(device, non_blocking=True)
# model = VGG16().to(device, non_blocking=True)
# torchsummary.summary(model, (3, 224, 224))

criterion = torch.nn.BCEWithLogitsLoss().to(device, non_blocking=True)
optimizer = torch.optim.Adam(model.parameters(), lr=args.learning_rate)

In [5]:
EXPERIMENT_DIR = f"./runs/{time.strftime('%Y-%m-%d-%H%M%S')}-{args.model_name}"
os.makedirs(EXPERIMENT_DIR, exist_ok=True)
tensorboard_writer = SummaryWriter(log_dir=EXPERIMENT_DIR)

train_losses, train_accuracys = [], []
val_losses, val_accuracys = [], []

for epoch in tqdm(range(args.epochs), total=args.epochs, desc="Epoch progress"):
    train_avg_loss, train_accuracy = 0.0, 0.0
    validation_avg_loss, validation_accuracy = 0.0, 0.0

    total_train_batch = len(train_dataloader)
    total_validation_batch = len(validation_dataloader)

    model.train()
    
    for datas, labels in tqdm(train_dataloader, total=total_train_batch, desc="train progress"):
        datas, labels = datas.float().to(device, non_blocking=True), labels.to(device, non_blocking=True)

        optimizer.zero_grad()
        predict = model(datas)
        loss = criterion(predict, labels)
        loss.backward()
        optimizer.step()
        train_avg_loss += loss.item() / total_train_batch
        train_accuracy += ((predict > 0.5) == labels).float().sum().item()

    train_accuracy /= len(train_dataset.data_paths)
    tensorboard_writer.add_scalar("Train Loss", train_avg_loss, epoch + 1)
    tensorboard_writer.add_scalar("Train Accuracy", train_accuracy, epoch + 1)
    
    train_losses.append(train_avg_loss)
    train_accuracys.append(train_accuracy)
    print("Epoch: ", "%d" % (epoch + 1), "train_loss: ", "{:.9f}".format(train_avg_loss), "train_accuracy: ", train_accuracy)

    with torch.no_grad():
        model.eval()

        for datas, labels in tqdm(validation_dataloader, total=total_validation_batch, desc="validation progress"):
            datas, labels = datas.float().to(device, non_blocking=True), labels.to(device, non_blocking=True)

            predict = model(datas)

            loss = criterion(predict, labels).detach()
            validation_avg_loss += loss.item() / total_validation_batch
            validation_accuracy += ((predict > 0.5) == labels).float().sum().item()

        validation_accuracy /= len(validation_dataset.data_paths)
        tensorboard_writer.add_scalar("Validation Loss", validation_avg_loss, epoch + 1)
        tensorboard_writer.add_scalar("Validation Accuracy", validation_accuracy, epoch + 1)
        
        val_losses.append(validation_avg_loss)
        val_accuracys.append(validation_accuracy)
        print("Epoch: ", "%d" % (epoch + 1), "validation_loss: ", "{:.9f}".format(validation_avg_loss), "validation_accuracy: ", validation_accuracy)
    
    tensorboard_writer.flush()

train_losses, train_accuracys = np.array(train_losses), np.array(train_accuracys)
val_losses, val_accuracys = np.array(val_losses), np.array(val_accuracys)
train_logs = np.stack([train_losses, train_accuracys, val_losses, val_accuracys], axis=1)

train_log_df = pd.DataFrame(train_logs, columns=["train_loss", "train_accuracy", "validation_loss", "validation_accuracy"])
train_log_df.to_csv(EXPERIMENT_DIR + "/train_log.csv", sep=",")

tensorboard_writer.close()

Epoch progress:   0%|          | 0/50 [00:00<?, ?it/s]